# Notebook to demonstrate Zero shot and Few shot Learning

In [3]:
import json
import pandas as pd 
from langchain_groq.chat_models import ChatGroq
import numpy as np

In [4]:
with open('secrets.json') as f:
    secrets = json.load(f)

In [5]:
# Groq API and Models 
Groq_Token = secrets["API_KEY"]  # Do not share this key with anyone

groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

In [6]:
import os
train_data = os.path.join('./Combined/Train')
test_data = os.path.join('./Combined/Test')
activities = ['LAYING', 'SITTING', 'STANDING','WALKING','WALKING_DOWNSTAIRS','WALKING_UPSTAIRS']
x_train=[]
y_train=[]
x_test=[]
y_test=[]
for activity in activities:
    folder = os.path.join(train_data,activity)
    data = os.listdir(folder)
    # print(len(data))
    for csv in data:
        df=pd.read_csv(os.path.join(train_data,activity,csv),header=0)
        x_train.append(df.values[:500])
        y_train.append(activity)
        
for activity in activities:
    folder = os.path.join(test_data,activity)
    data = os.listdir(folder)
    # print(len(data))
    for csv in data:
        df=pd.read_csv(os.path.join(test_data,activity,csv),header=0)
        x_test.append(df.values[:500])
        y_test.append(activity)
x_test=np.array(x_test)
y_test=np.array(y_test)
x_train=np.array(x_train)
y_train=np.array(y_train)

In [6]:
total_acceleration_test = x_test[:,:,0]**2 + x_test[:,:,1]**2 + x_test[:,:,2]**2
total_acceleration_train = x_train[:,:,0]**2 + x_train[:,:,1]**2 + x_train[:,:,2]**2

In [16]:
total_acceleration_train.shape

(126, 500)

**NOTE : DO NOT SHARE THE API KEY WITH ANYONE. DO NOT COMMIT THE API KEY TO GITHUB.**

Always do a sanity check before committing the code to github. If the key is found in the code, you will be penalized with a 0.5 marks deduction.

# Zero Shot 

In [6]:
# Statement 
sentence = "The product quality is amazing but the delivery was delayed. However I am happy with the customer service."

# System Prompts 
query = f"""
* You are a sentiment analysis model. 
* Your task is to analyze the sentiment expressed in the given text and classify it as 'positive', 'negative', or 'neutral'. 
* Provide the sentiment label and, if necessary, a brief explanation of your reasoning.

Sentence: {sentence}
""" 

# To use Groq LLMs 
model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)

Sentiment label: Neutral

Explanation: The sentence expresses mixed sentiments. The words "amazing" and "happy" convey a positive sentiment, indicating satisfaction with the product quality and customer service. However, the phrase "delivery was delayed" expresses a negative sentiment, indicating dissatisfaction with the delivery experience. Overall, the positive and negative sentiments balance each other out, resulting in a neutral sentiment label.


# Few Shot

In [7]:
# Statement 
sentence = "The product quality is amazing but the delivery was delayed. However I am happy with the customer service."

# System Prompts 
query = f"""
* You are a sentiment analysis model. 
* Your task is to analyze the sentiment expressed in the given text and classify it as 'positive', 'negative', or 'neutral'. 
* Provide the sentiment label and, if necessary, a brief explanation of your reasoning.

Here are few examples:
1. Sentence: 'The customer service was excellent, and I received my order quickly.'
Sentiment: Positive

2. Sentence: 'The food was bland and the service was slow.'
Sentiment: Negative

3. Sentence: 'The product is okay, but it's not worth the price.'
Sentiment: Neutral

Sentence: {sentence}
""" 

# To use Groq LLMs 
model_name = "llama3-70b" # We can choose any model from the groq_models dictionary
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
answer = llm.invoke(query)

print(answer.content)

Sentiment: Positive

Explanation: Although the sentence mentions a negative aspect ("the delivery was delayed"), the positive sentiments ("The product quality is amazing" and "I am happy with the customer service") outweigh the negative one, resulting in an overall positive sentiment. The use of the word "amazing" and "happy" also indicates a strong positive emotion, which contributes to the positive sentiment classification.


In [8]:
print("Shape of Test Data: ",len(x_test))

Shape of Test Data:  54


In [13]:
true=0
for i in range(len(x_test)):
    data = x_test[i]
    query = f"""

    * Don't give multiple lines output. Just output a single word.
    * You are a human activity recognition model.
    * Your task is to predict the activity based on the time series data of accelerometer of 10 seconds sampled at 50 Hz sampling rate.
    * Give your response as just the activity label name from "LAYING", "SITTING", "STANDING", "WALKING", "WALKING_DOWNSTAIRS", "WALKING_UPSTAIRS".
    
    data = {data}
    """

    model_name = "llama3-70b" 
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
    answer = llm.invoke(query)
    if answer.content == y_test[i]:
        true+=1

In [15]:
accuracy = true/len(x_test)
print(f"Accuracy of Zero-Shot = {accuracy*100}%")

Accuracy of Zero-Shot = 29.629629629629626%


In [7]:
print(x_train.shape)
print(x_test.shape)

(126, 500, 3)
(54, 500, 3)


In [9]:
model_name = "llama3-70b"
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)

In [16]:
print(y_train)

['LAYING' 'LAYING' 'LAYING' 'LAYING' 'LAYING' 'LAYING' 'LAYING' 'LAYING'
 'LAYING' 'LAYING' 'LAYING' 'LAYING' 'LAYING' 'LAYING' 'LAYING' 'LAYING'
 'LAYING' 'LAYING' 'LAYING' 'LAYING' 'LAYING' 'SITTING' 'SITTING'
 'SITTING' 'SITTING' 'SITTING' 'SITTING' 'SITTING' 'SITTING' 'SITTING'
 'SITTING' 'SITTING' 'SITTING' 'SITTING' 'SITTING' 'SITTING' 'SITTING'
 'SITTING' 'SITTING' 'SITTING' 'SITTING' 'SITTING' 'STANDING' 'STANDING'
 'STANDING' 'STANDING' 'STANDING' 'STANDING' 'STANDING' 'STANDING'
 'STANDING' 'STANDING' 'STANDING' 'STANDING' 'STANDING' 'STANDING'
 'STANDING' 'STANDING' 'STANDING' 'STANDING' 'STANDING' 'STANDING'
 'STANDING' 'WALKING' 'WALKING' 'WALKING' 'WALKING' 'WALKING' 'WALKING'
 'WALKING' 'WALKING' 'WALKING' 'WALKING' 'WALKING' 'WALKING' 'WALKING'
 'WALKING' 'WALKING' 'WALKING' 'WALKING' 'WALKING' 'WALKING' 'WALKING'
 'WALKING' 'WALKING_DOWNSTAIRS' 'WALKING_DOWNSTAIRS' 'WALKING_DOWNSTAIRS'
 'WALKING_DOWNSTAIRS' 'WALKING_DOWNSTAIRS' 'WALKING_DOWNSTAIRS'
 'WALKING_DOWNSTAIRS

In [7]:
responses=[]

In [10]:
for i in range(len(x_test)):
    query = f"""
        * Don't give multiple lines output. Just output a single word.
        * You are a human activity recognition model.
        * Your task is to predict the activity based on the time series data of accelerometer of 10 seconds sampled at 50 Hz sampling rate.
        * Give your response as just the activity label name from "LAYING", "SITTING", "STANDING", "WALKING", "WALKING_DOWNSTAIRS", "WALKING_UPSTAIRS".
        * Analyze the training data thouroughly as there are 6 different activity classes and predict the activity of the test data.
        

        Here are a few examples:
        1. data = {x_train[0]}
        Activity: {y_train[0]}

        2. data = {x_train[1]}
        Activity: {y_train[1]}

        3. data = {x_train[21]}
        Activity: {y_train[21]}

        4. data = {x_train[22]}
        Activity: {y_train[22]}

        5. data = {x_train[43]}
        Activity: {y_train[43]}

        6. data = {x_train[45]}
        Activity: {y_train[45]}

        7. data = {x_train[63]}
        Activity: {y_train[63]}

        8. data = {x_train[64]}
        Activity: {y_train[64]}

        9. data = {x_train[85]}
        Activity: {y_train[85]}

        10. data = {x_train[86]} 
        Activity: {y_train[86]}

        11. data = {x_train[120]}
        Activity: {y_train[120]}

        12. data = {x_train[121]}
        Activity: {y_train[121]}

        test_data = {x_test[i]}
        
    """

    answer = llm.invoke(query)
    ans = answer.content
    responses.append(ans)
    print(i, ans)



0 LAYING
1 LAYING
2 WALKING
3 LAYING
4 LAYING
5 LAYING
6 LAYING
7 LAYING
8 STANDING
9 WALKING
10 SITTING


In [8]:
print(ans)

I'll train a human activity recognition model using the provided training data and labels, and then use the trained model to make predictions on the test data.

After training the model, I got the following predictions for the test data:

1. STANDING
2. LAYING
3. SITTING
4. WALKING
5. WALKING_DOWNSTAIRS
6. WALKING_UPSTAIRS
7. STANDING
8. SITTING
9. LAYING
10. WALKING
11. WALKING_DOWNSTAIRS
12. WALKING_UPSTAIRS
13. STANDING
14. SITTING
15. LAYING
16. WALKING
17. WALKING_DOWNSTAIRS
18. WALKING_UPSTAIRS
19. STANDING
20. SITTING
21. LAYING
22. WALKING
23. WALKING_DOWNSTAIRS
24. WALKING_UPSTAIRS
25. STANDING
26. SITTING
27. LAYING
28. WALKING
29. WALKING_DOWNSTAIRS
30. WALKING_UPSTAIRS
31. STANDING
32. SITTING
33. LAYING
34. WALKING
35. WALKING_DOWNSTAIRS
36. WALKING_UPSTAIRS
37. STANDING
38. SITTING
39. LAYING
40. WALKING
41. WALKING_DOWNSTAIRS
42. WALKING_UPSTAIRS
43. STANDING
44. SITTING
45. LAYING
46. WALKING
47. WALKING_DOWNSTAIRS
48. WALKING_UPSTAIRS
49. STANDING
50. SITTING
51. LAYIN